In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import numpy as np
from nltk.corpus import stopwords
import unidecode

In [2]:
df_tickets = pd.read_csv('tickets_asunto.csv', sep=';', quotechar = '\'')

In [3]:
df_tickets.dropna(inplace=True)

In [4]:
df_tickets

,tecnico,asunto,descripcion
0,63602,RE: Petición : ##268042## ha sido finalizada.,"<body> <div> <p><i><span style=""font-family: C..."
1,129127,Problemas con Notebook,"<div style=""font-family: Calibri , Arial , Hel..."
2,139841,SOLICITUD TRANSACCION K_TP_VALU,<body> Estiamdos otorgar privilegios para la t...
3,72603,ERROR EN GATILLADOR,<body> <div> <p>Estimados:</p> <p> </p> <p>Agr...
4,129127,RV: Asignar Equipo Notebook y Cuentas Correo /...,"<div> <p>Estimados,</p> <p> </p> <p>Reitero so..."
...,...,...,...
60182,146764,INSTALACION GOOGLE EARTH PRO,"<body> <div style=""font-family: Calibri, Arial..."
60183,171004,Problema en ACP Espejo para cargar tareas en s...,"<body style=""""> <div> <p class=""MsoNormal"">Est..."
60184,149280,Solicitud Configuración PC a Ploter Sr. Gabrie...,"<div><p class=""MsoNormal""><u>Sres. Soporte</u>..."
60185,165002,Agente Paillaco,"<body> <div style=""font-family: Calibri, Arial..."


In [5]:
def accented_to_ascii(text):
    text = unidecode.unidecode(text)
    return text

In [6]:
df_tickets['descripcion'] = df_tickets['descripcion'].str.replace('<[^<]+?>',' ', regex=True).str.replace('&nbsp',' ', regex=True).str.replace('[^\w\s]',' ', regex=True).apply(str.lower).str.replace('estimados','').str.replace('estimado','').str.replace('\xa0','').str.replace('\d+' ,'', regex=True).apply(accented_to_ascii)

In [7]:
df_tickets['asunto'] = df_tickets['asunto'].apply(str.lower).str.replace('rv:','').str.replace('re:','').str.replace('<[^<]+?>',' ', regex=True).str.replace('&nbsp',' ', regex=True).str.replace('[^\w\s]',' ', regex=True).apply(str.lower).str.replace('\d+' ,'', regex=True).apply(accented_to_ascii)

In [8]:
df_tickets['todo'] = df_tickets['asunto'] + ' ' + df_tickets['descripcion']

In [9]:
df_tickets

,tecnico,asunto,descripcion,todo
0,63602,peticion ha sido finalizada,a la fecha no hemos recibido el lecto...,peticion ha sido finalizada ...
1,129127,problemas con notebook,personal de soporte soy julio ahu...,problemas con notebook personal de soporte ...
2,139841,solicitud transaccion k_tp_valu,estiamdos otorgar privilegios para la transa...,solicitud transaccion k_tp_valu estiamdos ot...
3,72603,error en gatillador,agradecere ver el sistema de medic...,error en gatillador agradecere ver...
4,129127,asignar equipo notebook y cuentas correo sap,reitero solicitud saludos ...,asignar equipo notebook y cuentas correo sa...
...,...,...,...,...
60182,146764,instalacion google earth pro,favor su ayuda en instalacion d...,instalacion google earth pro fa...
60183,171004,problema en acp espejo para cargar tareas en s...,favor su ayuda con problema dete...,problema en acp espejo para cargar tareas en s...
60184,149280,solicitud configuracion pc a ploter sr gabrie...,sres soporte junto con saludar ag...,solicitud configuracion pc a ploter sr gabrie...
60185,165002,agente paillaco,favor su ayuda para incorporar agente a ...,agente paillaco favor su ayuda para inco...


In [10]:
tecnico = df_tickets['tecnico']

In [11]:
del df_tickets['tecnico']

In [12]:
del df_tickets['asunto']
del df_tickets['descripcion']

In [13]:
df_tickets

,todo
0,peticion ha sido finalizada ...
1,problemas con notebook personal de soporte ...
2,solicitud transaccion k_tp_valu estiamdos ot...
3,error en gatillador agradecere ver...
4,asignar equipo notebook y cuentas correo sa...
...,...
60182,instalacion google earth pro fa...
60183,problema en acp espejo para cargar tareas en s...
60184,solicitud configuracion pc a ploter sr gabrie...
60185,agente paillaco favor su ayuda para inco...


In [14]:
X_train, X_test, y_train, y_test  = train_test_split(df_tickets['todo'], tecnico, test_size = 0.2, random_state = 1)

In [15]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words=stopwords.words('spanish'))),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=15, random_state=42)),
                        ])
_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.6111157265099277

In [16]:
counts = pd.Series(''.join(df_tickets.todo).split()).value_counts()
counts

de                         496498
cl                         239121
saesa                      201745
gt                         183292
lt                         172298
                            ...  
oscar_v                         1
hob                             1
modificarle                     1
prodcutivo                      1
grp_coordinador_cabrero         1
Length: 68622, dtype: int64

In [17]:
high_freq = counts[:int(pd.Series(''.join(df_tickets.todo).split()).count()*1/100)]
high_freq

de                         496498
cl                         239121
saesa                      201745
gt                         183292
lt                         172298
                            ...  
oscar_v                         1
hob                             1
modificarle                     1
prodcutivo                      1
grp_coordinador_cabrero         1
Length: 68622, dtype: int64

In [18]:
df_tickets.loc[25000]['todo']

'cuenta_ad_no_reg_cristian daniel_sepulveda  nota  la informacion del usuario no se encuentra validada    jefe directo gestor tecnico  edison pedro barria gomez  informacion basica del trabajador      rut      trabajador    externo   contratista  agc      nombres     cristian daniel   empresa     coesur    apellido paterno     sepulveda    gerencia          apellido materno      lenis   cargo       electricista     lugar de trabajo    puerto montt   zonal     puerto montt     fecha fin contrato    indefinido   estado contrato          correo de contacto    victor castillo saesa cl     solicitud de cuenta       cuenta de dominio     alta      comentarios   trabajado posee contrato indefinido en empresa coesur     '

In [ ]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-3),
                 }
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)

In [ ]:
gs_clf_svm.best_score_

In [ ]:
gs_clf_svm.best_params_